In [0]:
!unzip 'nlp assignment 3.zip'

Archive:  nlp assignment 3.zip
   creating: nlp assignment 3/
  inflating: nlp assignment 3/test.txt  
  inflating: nlp assignment 3/train.txt  


In [0]:
cd 'nlp assignment 3'

/content/nlp assignment 3


In [0]:
f = open("train.txt","r")
uncleaned = f.readlines()
train_data=[]

i=0
while(i<len(uncleaned)):
  x=uncleaned[i]
  l = x.split()
  if(l!=[]):
    if(l[0]=='meta'):
      d=dict()
      d['id']=int(l[1])
      d['sentiment']=str(l[2])
      d['tokens']=[]
      d['lang_id']=[]
    elif(len(l)==2):
      d['tokens'].append(l[0].lower())
      d['lang_id'].append(l[1])
  else:
    train_data.append(d)
  i=i+1

train_labels = []
for i in range(len(train_data)):
  s = train_data[i]['sentiment']
  train_labels.append(s)

In [0]:
f = open("test.txt","r")
uncleaned = f.readlines()
test_data=[]

i=0
while(i<len(uncleaned)):
  x=uncleaned[i]
  l = x.split()
  if(l!=[]):
    if(l[0]=='meta'):
      d=dict()
      d['id']=int(l[1])
      d['sentiment']=str(l[2])
      d['tokens']=[]
      d['lang_id']=[]
    elif(len(l)==2):
      d['tokens'].append(l[0].lower())
      d['lang_id'].append(l[1])
  else:
    test_data.append(d)
  i=i+1

test_labels = []
for i in range(len(test_data)):
  s = test_data[i]['sentiment']
  test_labels.append(s)

In [0]:
import nltk
nltk.download('punkt')
from nltk.util import ngrams

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
max_chars = 25
vector_size = 128

In [0]:
train_data_1 = []
for i in range(len(train_data)):
  t = train_data[i]['tokens']
  l = train_data[i]['lang_id']
  t_filter = [t[x] for x in range(len(t)) if l[x]!='O']
  t_trigram = [' '.join(t_filter[i:i+3]) for i in range(len(t_filter)-2)]
  #t_trigram += [' '.join(t_filter[i:i+4]) for i in range(len(t_filter)-3)]
  #t_trigram += [' '.join(t_filter[i:i+5]) for i in range(len(t_filter)-4)]
  #t_trigram += [' '.join(t_filter[i:i+6]) for i in range(len(t_filter)-5)]
  train_data_1.append(t_trigram)

In [0]:
a,b,c,d,e = 0,0,0,0,0
for i in range(len(train_data_1)):
  if(len(train_data_1[i])>40):
    a=a+1
  if(len(train_data_1[i])>50):
    b=b+1
  if(len(train_data_1[i])>75):
    c=c+1
  if(len(train_data_1[i])>90):
    d=d+1
  if(len(train_data_1[i])>100):
    e=e+1

In [0]:
print(a,b,c,d,e)

0 0 0 0 0


In [0]:
test_data_1 = []
for i in range(len(test_data)):
  t = test_data[i]['tokens']
  l = test_data[i]['lang_id']
  t_filter = [t[x] for x in range(len(t)) if l[x]!='O']
  t_trigram = [' '.join(t_filter[i:i+3]) for i in range(len(t_filter)-2)]
  #t_trigram += [' '.join(t_filter[i:i+4]) for i in range(len(t_filter)-3)]
  #t_trigram += [' '.join(t_filter[i:i+5]) for i in range(len(t_filter)-4)]
  #t_trigram += [' '.join(t_filter[i:i+6]) for i in range(len(t_filter)-5)]
  test_data_1.append(t_trigram)

In [0]:
train_labels_1 = []
for i in range(len(train_labels)):
  s = train_labels[i]
  if(s=='negative'):
    train_labels_1.append([1, 0, 0])
  elif(s=='positive'):
    train_labels_1.append([0, 0, 1])
  else:
    train_labels_1.append([0, 1, 0])

In [0]:
test_labels_1 = []
for i in range(len(test_labels)):
  s = test_labels[i]
  if(s=='negative'):
    test_labels_1.append([1, 0, 0])
  elif(s=='positive'):
    test_labels_1.append([0, 0, 1])
  else:
    test_labels_1.append([0, 1, 0])

In [0]:
import gensim
import numpy as np

In [0]:
y_train_1 = np.array(train_labels_1)
#y_test_1 = np.array(test_labels_1)

In [0]:
word_model_1 = gensim.models.Word2Vec(train_data_1+test_data_1, size=vector_size, seed=1011, min_count=1, iter=100)

In [0]:
word_vectors_1 = word_model_1.wv

In [0]:
vocab_size = word_model_1.wv.vectors.shape[0]
print(vocab_size)
embedding_size = word_model_1.wv.vectors.shape[1]
print(embedding_size)

275423
128


In [0]:
X_train_1 = np.zeros((len(train_data_1), max_chars, vector_size))
X_test_1 = np.zeros((len(test_data_1), max_chars, vector_size))

In [0]:
for i in range(len(train_data_1)):
  t = train_data_1[i][:max_chars]
  for j in range(len(t)):
    X_train_1[i, j, :] = word_vectors_1[t[j]]

for i in range(len(test_data_1)):
  t = test_data_1[i][:max_chars]
  for j in range(len(t)):
    X_test_1[i, j, :] = word_vectors_1[t[j]]

In [0]:
X_train_1.shape

(15131, 25, 128)

In [0]:
from keras.layers.recurrent import LSTM
#from keras.layers.embeddings import Embedding
from keras.layers import Bidirectional, Dense, Activation, SimpleRNN, Dropout
from keras.models import Sequential

In [0]:
model_1 = Sequential()
model_1.add(LSTM(units = 128, activation='relu', input_shape=(X_train_1.shape[1], X_train_1.shape[2]), return_sequences=False))
model_1.add(Dense(16, activation='relu'))
model_1.add(Dropout(0.10))
model_1.add(Dense(3, activation='softmax'))
model_1.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [0]:
batch_size = 32
nb_epochs = 10

In [0]:
model_1.fit(X_train_1, y_train_1,
          batch_size=batch_size,
          epochs=nb_epochs)

Epoch 1/10
15131/15131 [==============================] - 16s 1ms/step - loss: 1.0774 - acc: 0.4139
Epoch 2/10
15131/15131 [==============================] - 13s 892us/step - loss: 1.0437 - acc: 0.4418
Epoch 3/10
15131/15131 [==============================] - 13s 886us/step - loss: 1.0398 - acc: 0.4423
Epoch 4/10
15131/15131 [==============================] - 14s 894us/step - loss: 1.0331 - acc: 0.4459
Epoch 5/10
15131/15131 [==============================] - 13s 854us/step - loss: 1.0302 - acc: 0.4519
Epoch 6/10
15131/15131 [==============================] - 14s 893us/step - loss: 1.0266 - acc: 0.4548
Epoch 7/10
15131/15131 [==============================] - 14s 897us/step - loss: 1.0224 - acc: 0.4571
Epoch 8/10
15131/15131 [==============================] - 14s 903us/step - loss: 1.0216 - acc: 0.4580
Epoch 9/10
15131/15131 [==============================] - 16s 1ms/step - loss: 1.0216 - acc: 0.4560
Epoch 10/10
15131/15131 [==============================] - 14s 903us/step - loss: 1.01

In [0]:
model_1.fit(X_train_1, y_train_1,
          batch_size=batch_size,
          epochs=nb_epochs)

Epoch 1/10
15131/15131 [==============================] - 14s 896us/step - loss: 1.0134 - acc: 0.4640
Epoch 2/10
15131/15131 [==============================] - 13s 868us/step - loss: 1.0134 - acc: 0.4659
Epoch 3/10
15131/15131 [==============================] - 13s 874us/step - loss: 1.0147 - acc: 0.4585
Epoch 4/10
15131/15131 [==============================] - 15s 981us/step - loss: 1.0095 - acc: 0.4641
Epoch 5/10
15131/15131 [==============================] - 14s 933us/step - loss: 1.0064 - acc: 0.4636
Epoch 6/10
15131/15131 [==============================] - 13s 865us/step - loss: 1.0064 - acc: 0.4697
Epoch 7/10
15131/15131 [==============================] - 13s 842us/step - loss: 1.0226 - acc: 0.4663
Epoch 8/10
15131/15131 [==============================] - 14s 906us/step - loss: 1.0276 - acc: 0.4606
Epoch 9/10
15131/15131 [==============================] - 15s 986us/step - loss: 1.0381 - acc: 0.4588
Epoch 10/10
15131/15131 [==============================] - 14s 916us/step - loss: 

In [0]:
model_1.fit(X_train_1, y_train_1,
          batch_size=batch_size,
          epochs=nb_epochs)

Epoch 1/10
15131/15131 [==============================] - 13s 841us/step - loss: 1.0956 - acc: 0.3727
Epoch 2/10
15131/15131 [==============================] - 15s 986us/step - loss: 1.0941 - acc: 0.3726
Epoch 3/10
15131/15131 [==============================] - 14s 907us/step - loss: 1.0940 - acc: 0.3726
Epoch 4/10
15131/15131 [==============================] - 16s 1ms/step - loss: 1.0941 - acc: 0.3726
Epoch 5/10
15131/15131 [==============================] - 13s 864us/step - loss: 1.0941 - acc: 0.3726
Epoch 6/10
15131/15131 [==============================] - 13s 864us/step - loss: 1.0940 - acc: 0.3726
Epoch 7/10
15131/15131 [==============================] - 13s 853us/step - loss: 2.6655 - acc: 0.3725
Epoch 8/10
15131/15131 [==============================] - 13s 883us/step - loss: 9.1768 - acc: 0.3726
Epoch 9/10
15131/15131 [==============================] - 14s 925us/step - loss: 9.2273 - acc: 0.3726
Epoch 10/10
15131/15131 [==============================] - 14s 933us/step - loss: 9.

In [0]:
y_pred_1 = model_1.predict(X_test_1)

In [0]:
train_data_2 = []
for i in range(len(train_data)):
  t = train_data[i]['tokens']
  l = train_data[i]['lang_id']
  t_filter = [t[x] for x in range(len(t)) if l[x]!='O']
  t_trigram = t_filter + [' '.join(t_filter[i:i+2]) for i in range(len(t_filter)-1)]
  train_data_2.append(t_trigram)

In [0]:
test_data_2 = []
for i in range(len(test_data)):
  t = test_data[i]['tokens']
  l = test_data[i]['lang_id']
  t_filter = [t[x] for x in range(len(t)) if l[x]!='O']
  t_trigram = t_filter + [' '.join(t_filter[i:i+2]) for i in range(len(t_filter)-1)]
  test_data_2.append(t_trigram)

In [0]:
uni_bi_grams = []
for i in range(len(train_data_2)):
  uni_bi_grams.extend(train_data_2[i])
for i in range(len(test_data_2)):
  uni_bi_grams.extend(test_data_2[i])

uni_bi_grams = nltk.FreqDist(uni_bi_grams)

uni_bi_features = []
for gram in uni_bi_grams:
  if(uni_bi_grams[gram]>3):
    uni_bi_features.append(gram)

In [0]:
X_train_2 = []
for i in range(len(train_data_2)):
  f = {}
  for w in train_data_2[i]:
    f[w] = (w in uni_bi_features)
  X_train_2.append(f)

In [0]:
X_test_2 = []
for i in range(len(test_data_2)):
  f = {}
  for w in test_data_2[i]:
    f[w] = (w in uni_bi_features)
  X_test_2.append(f)

In [0]:
y_train_2 = np.array(train_labels)
#y_test_2 = np.array(test_labels)

In [0]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB

In [0]:
model_2 = SklearnClassifier(MultinomialNB())
model_2.train([(X_train_2[i], y_train_2[i]) for i in range(len(X_train_2))])

<SklearnClassifier(MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))>

In [0]:
model_2.labels()

['negative', 'neutral', 'positive']

In [0]:
v = model_2._vectorizer.transform(X_test_2)
y_pred_2 = model_2._clf.predict_proba(v)

In [0]:
y_pred = []
for i in range(len(y_pred_1)):
  p = []
  for j in range(3):
    p.append((y_pred_1[i][j])*(y_pred_2[i][j]))
  y_pred.append(np.array(p))

In [0]:
y_pred_labels = []
for i in range(len(y_pred)):
  x = y_pred[i].argmax()
  if(x==0):
    y_pred_labels.append('negative')
  elif(x==1):
    y_pred_labels.append('neutral')
  else:
    y_pred_labels.append('positive')

In [0]:
y_actual_labels = test_labels.copy()

In [0]:
count = 0
for i in range(len(y_pred_labels)):
  if(y_pred_labels[i]==y_actual_labels[i]):
    count=count+1
Accuracy = count/len(y_actual_labels)
print(Accuracy)

0.5446762974852862


In [0]:
from sklearn.metrics import f1_score
F1 = f1_score(y_actual_labels, y_pred_labels, average=None)
print(F1)

[0.57117595 0.47101449 0.60309698]


In [0]:
from sklearn.metrics import recall_score
Recall = recall_score(y_actual_labels, y_pred_labels, average=None)
print(Recall)

[0.60600375 0.43103448 0.63573883]


In [0]:
from sklearn.metrics import precision_score
Precision = precision_score(y_actual_labels, y_pred_labels, average=None)
print(Precision)

[0.54013378 0.51916933 0.57364341]
